In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from hazma.scalar_mediator import ScalarMediator
from hazma.vector_mediator import VectorMediator
from hazma.parameters import neutral_pion_mass as mpi0
from hazma.cmb import f_eff_ep, f_eff_g

In [ ]:
# Masses to scan over
mx_min, mx_max, n_mxs = mpi0/2., 250., 150
mxs = np.linspace(mx_min, mx_max, n_mxs)

Scalar mediator

In [ ]:
s_theta = 0.5
sm_hp = ScalarMediator(mx=1., ms=1000., gsxx=1.0, gsff=s_theta, gsGG=s_theta, gsFF=-s_theta)

sm_f_effs = sm_hp.f_effs(mxs)

Vector mediator

In [ ]:
vm = VectorMediator(mx=1., mv=1000., gvxx=1., gvuu=1., gvdd=0.2, gvss=0., gvee=0., gvmumu=0.)

vm_f_effs = vm.f_effs(mxs)

Plot together

In [ ]:
plt.figure(figsize=(5, 3.5))

plt.plot(mxs, sm_f_effs, label="Scalar")
plt.plot(mxs, vm_f_effs, label="Vector")

plt.xlim(mxs[[0, -1]])
plt.ylim(0, 1.)
plt.xlabel(r"$m_\chi$ (MeV)")
plt.ylabel(r"$f_{\mathrm{eff}}^\chi$")
plt.legend()

plt.savefig("figures/f_eff_dm.pdf")

$f_{\mathrm{eff}}$

In [ ]:
plt.plot(f_eff_ep.x, f_eff_ep.y, label=r"$f_{\mathrm{eff}}^{e^+ e^-}$")
plt.plot(f_eff_g.x, f_eff_g.y, label=r"$f_{\mathrm{eff}}^\gamma$")

plt.xlim([0.005, 250])
plt.xscale("log")
# plt.yscale("log")

# plt.xlim(mpi0/2., 250)

# plt.ylim(0, 2e2)
plt.xlabel(r"$E$ (MeV)")
plt.ylabel(r"$f_{\mathrm{eff}}$")
plt.legend()

In [ ]:
plt.plot(f_eff_ep.x, sm_hp.positron_spectra(f_eff_ep.x))